In [1]:
import numpy as np
import os
import pandas as pd
import h5py
from scipy.io import loadmat
import socket

hostname = socket.gethostname()
p = {}
if hostname == 'syndrome' or hostname == 'zod.psych.nyu.edu' or hostname == 'zod':
    p['datc'] = '/d/DATC/datc/MD_TMS_EEG'
else:
    p['datc'] = '/Users/mrugankdake/Documents/Clayspace/EEG_TMS/datc/MD_TMS_EEG'
p['data'] = p['datc'] + '/data'
p['analysis'] = p['datc'] + '/analysis'
p['EEGfiles'] = p['datc'] + '/EEGfiles'
p['meta'] = p['analysis'] + '/meta_analysis'
p['df_fname'] = os.path.join(p['meta'], 'calib_filtered.csv')
p['ALI_evoked'] = os.path.join(p['EEGfiles'], 'ALI_evoked.mat')
p['ALI_induced'] = os.path.join(p['EEGfiles'], 'ALI_induced.mat')
p['training_data'] = os.path.join(p['EEGfiles'], 'training_data.npy')
# Load up summary meta-data
summary_df = pd.read_csv(os.path.join(p['analysis'] + '/EEG_TMS_meta_Summary.csv'))
All_metadata = {row['Subject ID']: row for _, row in summary_df.iterrows()}

# Load up behavioral data
df_behav = pd.read_csv(p['df_fname'])

In [5]:
with h5py.File(p['ALI_evoked'], 'r') as f:
    X = np.array(f['mALI']).T
    y = np.array(f['trl_mat']).T

In [11]:
y_df = pd.DataFrame(y, columns = ['subjID', 'day', 'istms', 't_type', 'trl_idx'])
y_df.describe()

,0,1,2,3
count,20066.000000,20066.000000,20066.000000,20066.000000
mean,14.857271,0.664407,2.503588,200.228396
std,8.285075,0.472209,1.118368,114.994192
min,1.000000,0.000000,1.000000,1.000000
25%,7.000000,0.000000,2.000000,101.000000
50%,15.000000,1.000000,3.000000,200.000000
75%,23.000000,1.000000,4.000000,299.000000
max,27.000000,1.000000,4.000000,422.000000


In [5]:
sub_list = [1]
day_list = [1, 2, 3]
conditions = ['pin', 'pout', 'ain', 'aout']

data_dict = {cond: {ss: {dd: [] for dd in day_list} for ss in sub_list} for cond in conditions}

subject_day_info = []
freq_band = (8, 12)
time_band = (-1, 4.5)
ch_count = None
time_points = None
tr_count = 0

tfr_type = 'evoked'

if os.path.exists(p['training_data']):
    data_matrix = np.load(p['training_data'])
else:
    for cond in conditions:
        for ss in sub_list:
            for dd in day_list:
                this_fname = os.path.join(p['EEGfiles'], f'sub{ss:02}', f'day{dd:02}', f'sub{ss:02}_day{dd:02}_TFR_'+tfr_type+'.mat')
                with h5py.File(this_fname, 'r') as f:
                    # Load the time-frequency data
                    powspctrm = np.array(f['POW'][cond]['powspctrm'])
                    ch_labels = np.array(f['POW'][cond]['label']).astype(str)
                    # Create order of channel labels first time running this
                    if tr_count == 0:
                        standard_labels = ch_labels
                    
                    # Reorder data for channel indices are different from the one in first dataset
                    channel_indices = np.array([np.where(standard_labels == ch)[0][0] for ch in ch_labels])
                    powspctrm = powspctrm[:, :, channel_indices, :]

                    # Slice along the time of interest -1 to 4.5 seconds
                    time = np.array(f['POW'][cond]['time'])
                    time_band_indices = np.where((time >= time_band[0]) & (time <= time_band[1]))[0]
                    powspctrm = powspctrm[time_band_indices, :, :, :]

                    # Average over the alpha band
                    freqs = np.array(f['POW'][cond]['freq'])
                    freq_band_indices = np.where((freqs >= freq_band[0]) & (freqs <= freq_band[1]))[0]
                    powspctrm_avg = np.mean(powspctrm[:, freq_band_indices, :, :], axis=1)

                    # Reorder X_avg in the shape (trials, channels, time)
                    powspctrm_avg = np.transpose(powspctrm_avg, (2, 1, 0))

                    data_dict[cond][ss][dd] = powspctrm_avg

                    if ch_count is None:
                        ch_count = powspctrm_avg.shape[1]
                    if time_points is None:
                        time_points = powspctrm_avg.shape[2]
                    tr_count += powspctrm_avg.shape[0]

    data_matrix = np.zeros((len(conditions), len(sub_list), len(day_list), tr_count, ch_count, time_points))
    current_trial_index = 0

    for cond_idx, cond in enumerate(conditions):
        for ss_idx, ss in enumerate(sub_list):
            for dd_idx, dd in enumerate(day_list):
                data = data_dict[cond][ss][dd]
                num_trials = data.shape[0]
                data_matrix[cond_idx, ss_idx, dd_idx, current_trial_index:current_trial_index+num_trials, :, :] = data
                current_trial_index += num_trials

    np.save(p['training_data'], data_matrix)

(4, 1, 3, 1189, 62, 146)


In [ ]:
aa = np.transpose(X_avg, (2, 1, 0))
print(aa.shape)

In [ ]:
time.shape